In [17]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd

from torch import tensor

from generate_data import generate_data

# onehot = pd.Series(data = [tensor([1,0,0]),tensor([0,1,0]),tensor([0,0,1])], index = ['a','b','c'])
onehot = pd.Series(data = [[1,0,0],[0,1,0],[0,0,1]], index = ['a','b','c'])


In [5]:
N = 4*int(1e4)


X, y = generate_data(N, is_short=True, type = 2)


In [72]:
def encode_letter(l):
    return np.array([tensor(onehot[d].values) for d in l])

letterToIndex = lambda l: ord(l) - 97

def encode_language(l):
    tensor = torch.zeros(len(l), 1, 3)
    for li, letter in enumerate(l):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

def onehot_encode(data):
    return [encode_language(l) for l in data]

In [73]:
X_encoded = onehot_encode(X)

In [102]:
class LSTM_predictor(nn.Module):
    def __init__(self, SIZE):
        super(LSTM_predictor, self).__init__()

        self.SIZE = SIZE

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(3, 2)

        self.output = nn.Linear(2, 2)

        self.activation = nn.Softmax()

        # self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, sentence):

        lstm_out, _ = self.lstm(sentence)
        
        out = self.output(lstm_out)

        # return self.softmax(out)

        out = torch.sum(out,axis = 0)

        out = self.activation(out)

        return out
model = LSTM_predictor(64)

In [103]:
model(X_encoded[0])
# X_encoded[0]

c:\Users\jakob\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[0.3137, 0.6863]], grad_fn=<SoftmaxBackward>)